# Preparing the use case #

Create Table

In [0]:
-- 1- Create the credit cards table
CREATE TABLE credit_cards (
customer_id INT,
is_fraud BOOLEAN,
credit_card TEXT
);

Populate Table

In [0]:
-- 2- Populate the table with sample values
INSERT INTO credit_cards
VALUES
(100,'n', '453299ABCDEF4842'),
(100,'y', '471600ABCDEF5888'),
(102,'n', '524311ABCDEF2649'),
(102,'y', '601172ABCDEF4675'),
(102,'n', '601137ABCDEF9710'),
(103,'n', '373611ABCDEF6352')
;

# Creating the Solution #

Create Users and Roles with appropriate grants

In [0]:
--run GRANT to grant SELECT permission on the table
GRANT SELECT ON credit_cards TO PUBLIC;
--create four users
CREATE USER Kate WITH PASSWORD '1234Test!';
CREATE USER Ken  WITH PASSWORD '1234Test!';
CREATE USER Bob  WITH PASSWORD '1234Test!';
CREATE USER Jane WITH PASSWORD '1234Test!';


-- 1. Create User Roles
CREATE ROLE cust_srvc_role;
CREATE ROLE frdprvnt_role;
CREATE ROLE auditor_role;
-- note that public role exist by default.

-- Grant Roles to Users
GRANT ROLE cust_srvc_role to Kate;
GRANT ROLE frdprvnt_role  to Ken;
GRANT ROLE auditor_role   to Bob;
-- note that regualr_user is attached to public role by default.

Create Masking Functions and Policies

In [0]:
-- 2. Create Masking policies

-- 2.1 create a masking policy that fully masks the credit card number
CREATE MASKING POLICY Mask_CC_Full
WITH (credit_card VARCHAR(256))
USING ('XXXXXXXXXXXXXXXX'::text);     -- NOTE: Static masking expression require type casting 

--2.2- Create a scalar SQL user-defined function(UDF) that partially obfuscates credit card number, only showing the first 6 digits and the last 4 digits
CREATE FUNCTION REDACT_CREDIT_CARD (text)
  returns text
immutable
as $$
  select left($1,6)||'XXXXXX'||right($1,4)
$$ language sql;


--2.3- create a masking policy that applies the REDACT_CREDIT_CARD function
CREATE MASKING POLICY Mask_CC_Partial
WITH (credit_card VARCHAR(256))
USING (REDACT_CREDIT_CARD(credit_card));

-- 2.4- create a masking policy that will display raw credit card number only if it is flagged for fraud 
CREATE MASKING POLICY Mask_CC_Conditional
WITH (is_fraud BOOLEAN, credit_card VARCHAR(256))
USING (CASE WHEN is_fraud 
                 THEN credit_card 
                 ELSE Null 
       END);

-- 2.5- Create masking policy that will show raw credit card number.
CREATE MASKING POLICY Mask_CC_Raw
WITH (credit_card varchar(256))
USING (credit_card);

Attach Masking Policies on table/column to user/role

In [0]:
-- 3. ATTACHING MASKING POLICY
-- 3.1- make the Mask_CC_Full the default policy for all users
--    all users will see this masking policy unless a higher priority masking policy is attached to them or their role

ATTACH MASKING POLICY Mask_CC_Full
ON credit_cards(credit_card)
TO PUBLIC;

-- 3.2- attach Mask_CC_Partial to the cust_srvc_role role
--users with the cust_srvc_role role can see partial credit card information
ATTACH MASKING POLICY Mask_CC_Partial
ON credit_cards(credit_card)
TO ROLE cust_srvc_role
PRIORITY 10;

-- 3.3- Attach Mask_CC_Conditional masking policy to frdprvnt_role role
--    users with frdprvnt_role role can only see raw credit card if it is fraud
ATTACH MASKING POLICY Mask_CC_Conditional
ON credit_cards(credit_card)
USING (is_fraud, credit_card)
TO ROLE frdprvnt_role
PRIORITY 20;

-- 3.4- Attach Mask_CC_Raw masking policy to auditor_role role
--    users with auditor_role role can see raw credit card numbers
ATTACH MASKING POLICY Mask_CC_Raw
ON credit_cards(credit_card)
TO ROLE auditor_role
PRIORITY 30;

# Testing the Solution #

In [0]:
-- 1.1- Confirm the masking policies are created
SELECT * FROM svv_masking_policy;

-- 1.2- Verify attached masking policy on table/column to user/role.
SELECT * FROM svv_attached_masking_policy;

Test that customer service agents can only view the first 6 digits and the last 4 digits of the credit card number.

In [0]:
-- 1- Confirm that customer service agent can only view the first 6 digits and the last 4 digits of the credit card number
SET SESSION AUTHORIZATION Kate;
SELECT * FROM credit_cards;

Test that the Fraud Prevention users can only view raw credit card number when it is flagged as fraud.

In [0]:
-- 2- Confirm that Fraud Prevention users can only view fraudulent credit card number (conditional masking example)
SET SESSION AUTHORIZATION Ken;
SELECT * FROM credit_cards;

Test that Auditor users can view raw credit card number.

In [0]:
-- 3- Confirm the auditor can view RAW credit card number
SET SESSION AUTHORIZATION Bob;
SELECT * FROM credit_cards;

Test that General users can’t view any digit of the credit card number.

In [0]:
-- 4- Confirm that regular users can not view any digit of the credit card number
SET SESSION AUTHORIZATION Jane;
SELECT * FROM credit_cards;

# Modify the masking policy #

In [0]:
--reset session authorization to the default
RESET SESSION AUTHORIZATION;

Until now, Customer Service was able to see first 6 and last 4 digits of the credit card number.

We would want to make a policy change such that customer service can see last 4 digits only.

Let's create a NEW FUNCTION which only shows last 4 digits and redact remaining credit card number. In subsequent step, we will use it to alter the policy. 

**NOTE** *- You cannot drop or replace a function attached to a policy.*

In [0]:
-- Create a user-defined function that partially obfuscates credit card number, only showing the last 4 digits
CREATE FUNCTION REDACT_CREDIT_CARD_LAST4ONLY (credit_card TEXT) RETURNS TEXT IMMUTABLE AS $$
    import re
    regexp = re.compile("^([0-9A-F]{6})[0-9A-F]{5,6}([0-9A-F]{4})")
    match = regexp.search(credit_card)
    if match != None:
        last = match.group(2)
    else:
        last = "0000"
    return "XXXXXXXXXXXX{}".format(last)
$$ LANGUAGE plpythonu;

Execute ALTER MASKING POLICY statement

In [0]:
ALTER MASKING POLICY Mask_CC_Partial 
USING (REDACT_CREDIT_CARD_LAST4ONLY(credit_card));

Test that customer service agents can only view the last 4 digits of the credit card number.

In [0]:
-- Confirm that customer service agent can only view the last 4 digits of the credit card number
SET SESSION AUTHORIZATION Kate;
SELECT * FROM credit_cards;

# Lookup table based masking policy #

In [0]:
--reset session authorization to the default
RESET SESSION AUTHORIZATION;

Create a Lookup Table, and sample records in lookup

In [0]:
-- 1- Create the credit cards fraud lookup table
CREATE TABLE credit_cards_fraud_lookup (
credit_card_lookup TEXT
);


-- 2- Populate the table with sample values
INSERT INTO credit_cards_fraud_lookup
VALUES
('471600ABCDEF5888'),
('524311ABCDEF2649'),   --- note, this one had is_fraud flag n in main table
('601172ABCDEF4675');

Create a masking policy referencing a lookup table. 

Example: The fraud prevention policy could have been written as - If the credit card lies in the credit_card_fraud_lookup table, display full else display null.

In [0]:
-- create masking policy referencing lookup table
CREATE MASKING POLICY lookup_mask_credit_card WITH (credit_card TEXT) USING (
  CASE
    WHEN
      credit_card IN (SELECT credit_card_lookup FROM credit_cards_fraud_lookup) 
    THEN credit_card
    ELSE Null
    END
  );

GRANT SELECT ON TABLE credit_cards_fraud_lookup TO MASKING POLICY lookup_mask_credit_card;   ---- Provides access to lookup table via policy attached to role

Detach previous conditional mask policy from fraud prevention role and attach the new lookup masking policy.

In [0]:
--detach previous conditional masking policy from frdprvnt_role
DETACH MASKING POLICY Mask_CC_Conditional
ON                    credit_cards(credit_card)
FROM ROLE             frdprvnt_role;

-- attach lookup table based masking policy to frdprvnt_role
ATTACH MASKING POLICY lookup_mask_credit_card
ON credit_cards(credit_card)
USING (credit_card)
TO ROLE frdprvnt_role
PRIORITY 20;

Test that the Fraud Prevention users can only view raw credit card number which were present in lookup table.

In [0]:
-- Confirm that Fraud Prevention users can only view fraudulent credit card number (conditional masking via LOOKUP TABLE example)
SET SESSION AUTHORIZATION Ken;
SELECT * FROM credit_cards;

# Cleanup #

In [0]:
-- Cleanup

-- Revert current_user to admin or role sys:secadmin
RESET SESSION AUTHORIZATION;

--1.	Detach the masking policies from table
DETACH MASKING POLICY Mask_CC_Full
ON credit_cards(credit_card)
FROM PUBLIC;
DETACH MASKING POLICY Mask_CC_Partial
ON credit_cards(credit_card)
FROM ROLE cust_srvc_role;
DETACH MASKING POLICY lookup_mask_credit_card
ON credit_cards(credit_card)
FROM ROLE frdprvnt_role;
DETACH MASKING POLICY Mask_CC_Raw
ON credit_cards(credit_card)
FROM ROLE auditor_role;


-- 2.	Drop the masking policies 
DROP MASKING POLICY Mask_CC_Full;
DROP MASKING POLICY Mask_CC_Partial;
DROP MASKING POLICY Mask_CC_Conditional;
DROP MASKING POLICY Mask_CC_Raw;
DROP MASKING POLICY lookup_mask_credit_card;


-- 3.	Revoke/Drop - role/user 
REVOKE ROLE cust_srvc_role from Kate;
REVOKE ROLE frdprvnt_role  from Ken;
REVOKE ROLE auditor_role   from Bob;

DROP ROLE cust_srvc_role;
DROP ROLE frdprvnt_role;
DROP ROLE auditor_role;

DROP USER Kate;
DROP USER Ken;
DROP USER Bob;
DROP USER Jane;

-- 4.	Drop function and table 
DROP FUNCTION REDACT_CREDIT_CARD (credit_card TEXT);
DROP FUNCTION REDACT_CREDIT_CARD_LAST4ONLY (credit_card TEXT);
DROP TABLE credit_cards;
DROP TABLE credit_cards_fraud_lookup;

# USE CASE - 2 (Refer: Call to Action: SSN and EMAIL) #

Create table and populate it

In [0]:
CREATE TABLE customer (
customer_id INT,
e_mail TEXT,
SSN TEXT
);
-- 2- Populate the table with sample values with last entry with invalid e-mail
INSERT INTO customer
VALUES
(100,'customer1@abc.com', '111-11-1111'),
(101,'customer2@xyz.com', '222-22-2222'),
(102,'customer3@abcxyz.com', '333-33-3333'),
(103,'customer4@abc123.com', '444-44-4444'),
(104,'customer5@axyz.com', '555-55-5555'),
(105,'customer6@abc.com', '666-66-6666'),
(106,'customer7abc.com', '666-66-6666')
;

GRANT SELECT ON customer TO PUBLIC;

Create users

In [0]:
-- 3- create four users
CREATE USER cust_srvc_user WITH PASSWORD '1234Test!';
CREATE USER auditor_user   WITH PASSWORD '1234Test!';
CREATE USER regular_user   WITH PASSWORD '1234Test!';

Create Roles

In [0]:
-- Create User Roles
CREATE ROLE cust_srvc_role;
CREATE ROLE auditor_role;
-- note that public role exist by default

Grant roles to Users

In [0]:
-- 5- Grant Roles to Users
GRANT ROLE cust_srvc_role to cust_srvc_user;
GRANT ROLE auditor_role   to auditor_user;
-- note that regualr_user is attached to public role by default.

Create Masking Function

In [0]:
-- Create UDF for masking e-mail
CREATE OR REPLACE FUNCTION REDACT_EMAIL (email_address TEXT)
RETURNS TEXT IMMUTABLE
AS $$
    md=email_address.find('@')
    mdot=email_address.rfind('.')
    ln=len(email_address)
    if md>0 and mdot> 0 and ln>0:
            rtn_val = email_address[0]+'#####'+email_address[md-1:md+1]+'####' +email_address[mdot:ln]  
    else:
        rtn_val='invalid email'

    return rtn_val
$$ LANGUAGE plpythonu;


-- Create UDF for masking ssn

CREATE OR REPLACE FUNCTION REDACT_SSN (ssn TEXT)
RETURNS TEXT IMMUTABLE
AS $$
    import re
    vSSN = ''.join(re.findall(r'\d+',ssn ))
    if len(vSSN)==9:
        rtn_val = vSSN[0:3]+'-xx-xxxx'
    else:
        rtn_val='invalid ssn'

    return rtn_val
$$ LANGUAGE plpythonu;

Create Masking Policy

In [0]:
CREATE MASKING POLICY mask_e_mail_full
WITH (e_mail VARCHAR(256))
USING ('XXXXXXXX@XXXXXXXX.XXX');

ATTACH MASKING POLICY mask_e_mail_full
ON customer(e_mail)
TO PUBLIC;

CREATE MASKING POLICY e_mail_partial_mask
WITH (e_mail VARCHAR(256))
USING (REDACT_EMAIL(e_mail));

ATTACH MASKING POLICY e_mail_partial_mask
ON customer(e_mail)
USING (e_mail)
TO ROLE cust_srvc_role
PRIORITY 10;

CREATE MASKING POLICY raw_e_mail
WITH (e_mail VARCHAR(256))
USING (e_mail);

ATTACH MASKING POLICY raw_e_mail
ON customer(e_mail)
TO ROLE auditor_role
PRIORITY 20;


CREATE MASKING POLICY mask_ssn_full
WITH (SSN VARCHAR(256))
USING ('XXX-XX-XXXX');

ATTACH MASKING POLICY mask_ssn_full
ON customer(SSN)
TO PUBLIC;

CREATE MASKING POLICY ssn_partial_mask
WITH (SSN VARCHAR(256))
USING (REDACT_SSN(SSN));

ATTACH MASKING POLICY ssn_partial_mask
ON customer(SSN)
USING (SSN)
TO ROLE cust_srvc_role
PRIORITY 10;

CREATE MASKING POLICY raw_ssn
WITH (SSN VARCHAR(256))
USING (SSN);

ATTACH MASKING POLICY raw_ssn
ON customer(SSN)
TO ROLE auditor_role
PRIORITY 20;

Test for cust_srvc_user, should show patially masked ssn & email

In [0]:
SET SESSION AUTHORIZATION cust_srvc_user;
SELECT * FROM customer;

Test for Auditor_user, should show raw ssn & email

In [0]:
SET SESSION AUTHORIZATION auditor_user;
SELECT * FROM customer;

### Use case -2 Cleaup ###

In [0]:
SET SESSION AUTHORIZATION default;

Detach Masking Policy

In [0]:
DETACH MASKING POLICY mask_e_mail_full
ON customer(e_mail)
FROM PUBLIC;

DROP MASKING POLICY mask_e_mail_full;

DETACH MASKING POLICY e_mail_partial_mask
ON customer(e_mail)
FROM ROLE cust_srvc_role;

DROP  MASKING POLICY e_mail_partial_mask;

DETACH MASKING POLICY raw_e_mail
ON customer(e_mail)
FROM ROLE auditor_role;

DROP MASKING POLICY raw_e_mail;

DETACH MASKING POLICY mask_ssn_full
ON customer(SSN)
FROM PUBLIC;

DROP MASKING POLICY mask_ssn_full;

DETACH MASKING POLICY ssn_partial_mask
ON customer(SSN)
FROM ROLE cust_srvc_role;

DROP MASKING POLICY ssn_partial_mask;
DETACH MASKING POLICY raw_ssn
ON customer(SSN)
FROM ROLE auditor_role;

DROP MASKING POLICY raw_ssn;

Revoke role, drop functions

In [0]:
REVOKE ROLE cust_srvc_role from cust_srvc_user;
REVOKE ROLE auditor_role   from auditor_user;

DROP FUNCTION REDACT_EMAIL (email_address TEXT);
DROP FUNCTION REDACT_SSN (ssn TEXT);

Drop roles, user and table

In [0]:
DROP ROLE cust_srvc_role;
DROP ROLE auditor_role;

DROP USER cust_srvc_user ;
DROP USER auditor_user;

DROP TABLE customer;